# Vizard v2 Comprehensive Test Suite - Minimal Version

**Sections Included:**
1. Setup & Configuration
2. Dataset Loading & Exploration
3. Core Visual Encodings
4. Range Encodings (X2, Y2)

**Datasets:** CARS, STOCKS, MOVIES from altair.datasets

## 1. Setup & Configuration

### Load imports

In [ ]:
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import pandas as pd
import numpy as np
from altair.datasets import data

### Configure theme

In [ ]:
alt.renderers.enable('html')

@alt.theme.register('bioinformatics_theme', enable=True)
def bioinformatics_theme():
    return alt.theme.ThemeConfig({'width': 600, 'height': 400})

### Load Claude Code magic

In [ ]:
%load_ext vizard_magic

### Verify Vizard is working

In [ ]:
%cc HELP

**Expected:** Display Vizard help documentation with all keywords listed

## 2. Dataset Loading & Exploration

### Dataset 1: CARS

In [ ]:
cars_df = pl.DataFrame(data.cars())
print(f'Shape: {cars_df.shape}')
print(f'Columns: {cars_df.columns}')
cars_df.head(5)

### Dataset 2: STOCKS

In [ ]:
stocks_df = pl.DataFrame(data.stocks())
print(f'Shape: {stocks_df.shape}')
print(f'Columns: {stocks_df.columns}')
stocks_df.head(5)

### Dataset 3: MOVIES

In [ ]:
movies_df = pl.DataFrame(data.movies())
print(f'Shape: {movies_df.shape}')
print(f'Columns: {movies_df.columns}')
movies_df.head(5)

## 3. Core Visual Encodings

Test X, Y, COLOR, SIZE, SHAPE, OPACITY, ROW, COLUMN using CARS dataset.

### Test 3.1: Basic scatter (X, Y)

In [ ]:
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon

**Expected:** Basic scatter plot showing horsepower vs fuel efficiency

### Test 3.2: Add COLOR encoding

In [ ]:
%cc COLOR Origin

**Expected:** Points colored by car origin (USA, Europe, Japan)

### Test 3.3: Add SIZE encoding

In [ ]:
%cc SIZE Weight_in_lbs

**Expected:** Point sizes vary by car weight

### Test 3.4: Test SHAPE with COLOR

In [ ]:
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLOR Origin SHAPE Origin

**Expected:** Different shapes AND colors for each origin

### Test 3.5: OPACITY encoding

In [ ]:
%cc OPACITY Acceleration

**Expected:** Point transparency varies by acceleration

### Test 3.6: ROW faceting

In [ ]:
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon ROW Cylinders

**Expected:** Multiple scatter plots arranged horizontally, one per cylinder count

### Test 3.7: COLUMN faceting

In [ ]:
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLUMN Origin

**Expected:** Three scatter plots stacked vertically, one per origin

### Test 3.8: Complex - ROW + COLUMN + COLOR

In [ ]:
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLOR Origin ROW Cylinders COLUMN Origin

**Expected:** Multiple scatter plots arranged horizontally, one per cylinder count

## 4. Range Encodings (X2, Y2)

Test range encodings for error bars and intervals.

### Prepare aggregated data with ranges

In [ ]:
# Aggregate cars by Origin AND Cylinders for more bars (10-12 bars)
cars_agg = (
    cars_df
    .filter(pl.col('Cylinders').is_in([4, 6, 8]))  # Focus on common cylinder counts
    .group_by(['Origin', 'Cylinders'])
    .agg([
        pl.col('Miles_per_Gallon').mean().alias('avg_mpg'),
        pl.col('Miles_per_Gallon').min().alias('min_mpg'),
        pl.col('Miles_per_Gallon').max().alias('max_mpg')
    ])
    .with_columns([
        (pl.col('Origin') + ' ' + pl.col('Cylinders').cast(str) + 'cyl').alias('category')
    ])
    .sort(['Origin', 'Cylinders'])
)
print('Aggregated data with ranges by Origin + Cylinders:')
cars_agg

### Test 4.1: Vertical ranges with Y and Y2

In [ ]:
%cc RESET
%cc DATA cars_agg PLOT bar X category Y min_mpg Y2 max_mpg TITLE MPG Ranges by Origin and Cylinders

**Expected:** 9-10 bars showing MPG range (min to max) for each Origin+Cylinder combination

### Test 4.2: Error bars with point + rule

In [ ]:
# Create error bar visualization using natural language
%cc Create a chart with points at avg_mpg and error bars from min_mpg to max_mpg

**Expected:** Points with vertical error bars showing full MPG range

### Test 4.3: Range encoding with COLOR

In [ ]:
%cc COLOR Origin

**Expected:** Colored ranges, one color per origin

## 5. Advanced Encodings (SERIES, TEXT)

Test SERIES for multi-series line charts and TEXT for labels.

### Test 5.1: Multi-series line chart with SERIES

In [ ]:
%cc RESET
%cc DATA stocks PLOT line X date Y price SERIES symbol COLOR symbol

**Expected:** Four separate lines (MSFT, AAPL, GOOG, IBM), SERIES groups without adding visual encoding

### Test 5.2: Bar chart with TEXT labels

In [ ]:
# Aggregate cars by origin for simpler visualization
cars_by_origin = (
    cars_df.group_by('Origin')
    .agg(pl.col('Miles_per_Gallon').mean().alias('avg_mpg'))
)
%cc DATA cars_by_origin PLOT bar X Origin Y avg_mpg TEXT avg_mpg

**Expected:** Bar chart with MPG values displayed on top of each bar

### Test 5.3: Scatter with TEXT for high-rated movies

In [ ]:
# Filter top-rated movies with valid ratings
top_movies = (
    movies_df
    .filter(pl.col('IMDB Rating').is_not_null())
    .filter(pl.col('IMDB Rating') >= 8.0)
    .sort('IMDB Rating', descending=True)
    .head(15)
    .rename({'IMDB Rating': 'IMDB_Rating', 'IMDB Votes': 'IMDB_Votes'})
)
print(f'Top {len(top_movies)} movies:')
top_movies.select(['Title', 'IMDB_Rating'])

In [ ]:
%cc DATA top_movies PLOT scatter X IMDB_Rating Y IMDB_Votes TEXT Title TITLE Top Rated Movies

**Expected:** Scatter plot with movie titles labeling each point

### Test 5.4: Complex - SERIES + COLOR + layering

In [ ]:
%cc DATA stocks PLOT line X date Y price SERIES symbol COLOR symbol TITLE Stock Prices Over Time

**Expected:** Multi-line chart with distinct colors per stock symbol

## 6. Layout & Grouping (BAR_LAYOUT)

Test stacked, grouped, and normalized bar layouts.

### Prepare aggregated data by Origin and Cylinders

In [ ]:
# Get counts by Origin and Cylinders
cars_counts = (
    cars_df
    .filter(pl.col('Cylinders').is_in([4, 6, 8]))
    .group_by(['Origin', 'Cylinders'])
    .agg(pl.len().alias('count'))
)
print('Car counts by Origin and Cylinders:')
cars_counts

### Test 6.1: Stacked bars

In [ ]:
%cc RESET
%cc DATA cars_counts PLOT bar X Origin Y count COLOR Cylinders BAR_LAYOUT stacked

**Expected:** Bars stacked vertically, colored by cylinder count

### Test 6.2: Grouped bars (side-by-side)

In [ ]:
%cc BAR_LAYOUT grouped

**Expected:** Bars arranged side-by-side using xOffset

### Test 6.3: Normalized bars (100% stacked)

In [ ]:
%cc BAR_LAYOUT normalized

**Expected:** All bars reach 100%, showing proportions

### Test 6.4: Complex - Grouped bars with custom dimensions

In [ ]:
%cc BAR_LAYOUT grouped WIDTH 800 HEIGHT 500 TITLE Car Distribution by Origin and Cylinders

**Expected:** Grouped bars with larger dimensions and title

## 7. Data Transformations (WINDOW)

Test window transformations for running calculations.

### Test 7.1: Cumulative sum with STOCKS

In [ ]:
%cc RESET
%cc DATA stocks PLOT line X date Y price WINDOW cumsum SERIES symbol COLOR symbol TITLE Cumulative Stock Prices

**Expected:** Lines showing cumulative sum of prices over time, one per symbol

### Test 7.2: Ranking movies by rating

In [ ]:
%cc DATA top_movies PLOT bar X Title Y IMDB_Rating WINDOW rank TITLE Top Movies Ranked

**Expected:** Bar chart with ranking applied via transform_window

### Test 7.3: Row number

In [ ]:
%cc WINDOW row_number

**Expected:** Sequential numbering applied to data

### Test 7.4: Complex - WINDOW with SERIES (grouped cumsum)

In [ ]:
%cc DATA stocks PLOT line X date Y price WINDOW cumsum SERIES symbol COLOR symbol

**Expected:** Separate cumulative sums for each stock symbol

## 8. Meta Commands

Test state management commands: KEYWORDS, KEYS, RESET, HELP

### Test 8.1: KEYWORDS - show current state

In [ ]:
%cc KEYWORDS

**Expected:** Display all current keyword values from state

### Test 8.2: KEYS - alias for KEYWORDS

In [ ]:
%cc KEYS

**Expected:** Same output as KEYWORDS

### Test 8.3: RESET - clear state

In [ ]:
%cc RESET

**Expected:** State cleared, confirmation message

### Test 8.4: HELP - display documentation

In [ ]:
%cc HELP

**Expected:** Help text with all keyword definitions

### Test 8.5: Verify RESET worked

In [ ]:
%cc KEYS

**Expected:** Only default values shown (ENGINE: altair, WIDTH: 600, etc.)

## 9. Code Generation Options

Test FUNCTION and IMPORT keywords for code output control.

### Test 9.1: IMPORT true - generate imports

In [ ]:
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon IMPORT

**Expected:** Code starts with import statements (import altair as alt, import polars as pl)

### Test 9.2: IMPORT false (default) - no imports

In [ ]:
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon

**Expected:** Code starts directly with data loading, no imports

### Test 9.3: FUNCTION - generate reusable function

In [ ]:
%cc DATA cars PLOT bar X Origin Y Miles_per_Gallon FUNCTION IMPORT

**Expected:** Function definition with type hints, docstring, parameters, return type

### Test 9.4: FUNCTION + IMPORT together

In [ ]:
%cc DATA stocks PLOT line X date Y price SERIES symbol FUNCTION IMPORT

**Expected:** Complete function with imports, ready to use

## 10. State Management & Persistence

Test that state accumulates, persists, and can be cleared.

### Test 10.1: Set custom defaults without generating chart

In [ ]:
%cc RESET
%cc WIDTH 800 HEIGHT 600 DATA cars

**Expected:** No chart generated (no PLOT specified), but state saved

### Test 10.2: Generate chart with persisted state

In [ ]:
%cc PLOT scatter X Horsepower Y Miles_per_Gallon

**Expected:** Chart uses WIDTH: 800, HEIGHT: 600 from previous state

### Test 10.3: Add COLOR - state accumulates

In [ ]:
%cc COLOR Origin

**Expected:** Chart regenerated with color added, all previous state preserved

### Test 10.4: Verify accumulated state

In [ ]:
%cc KEYS

**Expected:** Shows WIDTH: 800, HEIGHT: 600, DATA: cars, PLOT, X, Y, COLOR

### Test 10.5: RESET clears all

In [ ]:
%cc RESET

**Expected:** State cleared

### Test 10.6: Verify defaults restored

In [ ]:
%cc KEYS

**Expected:** Only defaults (WIDTH: 600, HEIGHT: 400, ENGINE: altair, etc.)

### Test 10.7: Dynamic keyword - THRESHOLD

In [ ]:
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon THRESHOLD 100
%cc Color points where Horsepower > THRESHOLD in red, others in gray

**Expected:** Conditional coloring using custom THRESHOLD keyword

## 11. Syntax Flexibility

Test keyword-only, natural language, and mixed syntax styles.

### Test 11.1: Keywords only

In [ ]:
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon

**Expected:** Basic scatter from pure keyword syntax

### Test 11.2: Natural language

In [ ]:
%cc Create a scatter plot from cars dataset showing horsepower vs fuel efficiency

**Expected:** Same chart inferred from natural language

### Test 11.3: Mixed syntax

In [ ]:
%cc DATA stocks - create a line chart with X date Y price colored by symbol

**Expected:** Line chart combining keywords and natural language

### Test 11.4: VZ trigger

In [ ]:
%cc VZ DATA stocks PLOT line X date Y price SERIES symbol COLOR symbol

**Expected:** Multi-line chart using short trigger

### Test 11.5: VIZARD trigger

In [ ]:
%cc VIZARD make a histogram of Horsepower from cars dataset

**Expected:** Histogram using explicit trigger

### Test 11.6: Conversational refinement

In [ ]:
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon

In [ ]:
%cc COLOR Origin

In [ ]:
%cc SIZE Weight_in_lbs

In [ ]:
%cc TITLE Car Performance Analysis

**Expected:** Chart progressively enhanced through multiple cells

## 12. Edge Cases & Complex Scenarios

Stress test features, test combinations, find edge cases.

### Test 12.1: Null handling - movies with missing ratings

In [ ]:
# Check null distribution
print('Movies with null IMDB ratings:', movies_df.filter(pl.col('IMDB Rating').is_null()).shape[0])
%cc RESET
%cc DATA movies PLOT scatter X Production_Budget Y IMDB_Rating TITLE Budget vs Rating

**Expected:** Chart handles nulls gracefully (may show warnings or filter nulls)

### Test 12.2: Many simultaneous encodings

In [ ]:
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLOR Origin SIZE Weight_in_lbs OPACITY Acceleration

**Expected:** Complex scatter with X, Y, COLOR, SIZE, OPACITY all working together

### Test 12.3: Long column names

In [ ]:
%cc DATA movies PLOT scatter X Production_Budget Y Rotten_Tomatoes_Rating

**Expected:** Handles underscore-separated column names correctly

### Test 12.4: Datetime encoding

In [ ]:
%cc DATA stocks PLOT line X date Y price SERIES symbol

**Expected:** Date column recognized as temporal type (:T)

### Test 12.5: Very wide chart

In [ ]:
%cc WIDTH 1200 HEIGHT 300

**Expected:** Chart regenerated with panoramic aspect ratio

### Test 12.6: Multiple dynamic keywords

In [ ]:
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon THRESHOLD_LOW 15 THRESHOLD_HIGH 25
%cc Color points: red if MPG > THRESHOLD_HIGH, blue if MPG < THRESHOLD_LOW, gray otherwise

**Expected:** Multi-condition coloring using two custom keywords

### Test 12.7: Spelling tolerance

In [ ]:
%cc RESET
%cc DATA cars PLOT scatter X Horsepower Y Miles_per_Gallon COLOUR Origin TITEL Car Analysis HIGHT 450

**Expected:** Typos corrected (COLOUR→COLOR, TITEL→TITLE, HIGHT→HEIGHT)

### Test 12.8: ENGINE switching - matplotlib

In [ ]:
%cc ENGINE matplotlib

**Expected:** Same chart rendered with matplotlib instead of altair

### Test 12.9: ENGINE switching - seaborn

In [ ]:
%cc ENGINE seaborn

**Expected:** Chart rendered with seaborn styling

### Test 12.10: Complex WINDOW - grouped by symbol

In [ ]:
%cc ENGINE altair
%cc DATA stocks PLOT line X date Y price WINDOW cumsum SERIES symbol COLOR symbol TITLE Cumulative Price by Symbol

**Expected:** Each stock symbol has its own cumulative sum line

## Test Summary

**Sections tested:**
1. ✅ Setup & Configuration
2. ✅ Dataset Loading (CARS, STOCKS, MOVIES)
3. ✅ Core Visual Encodings (X, Y, COLOR, SIZE, SHAPE, OPACITY, ROW, COLUMN)
4. ✅ Range Encodings (X2, Y2)
5. ✅ Advanced Encodings (SERIES, TEXT)
6. ✅ Layout & Grouping (BAR_LAYOUT: stacked, grouped, normalized)
7. ✅ Data Transformations (WINDOW: cumsum, rank, row_number)
8. ✅ Meta Commands (KEYWORDS, KEYS, RESET, HELP)
9. ✅ Code Generation (FUNCTION, IMPORT)
10. ✅ State Management & Persistence
11. ✅ Syntax Flexibility (keywords, natural language, mixed)
12. ✅ Edge Cases (nulls, many encodings, datetime, ENGINE switching)

**Total test scenarios:** ~60+